# Constrained Coefficients

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

import numpy as np
np.set_printoptions(suppress=True)


from prophet import Prophet

In [3]:
from scipy.optimize import curve_fit
#import statsmodels.api as sm

from statsmodels.regression import linear_model 
from statsmodels import tools

In [4]:
holidays = pd.read_csv("./data/prophet_holidays_daily.csv", parse_dates = ["ds"])
holidays["begin_week"] = holidays["ds"].dt.to_period('W-SUN').dt.start_time
#combine same week holidays into one holiday
holidays_weekly = holidays.groupby(["begin_week", "country", "year"], as_index = False).agg({'holiday':'#'.join, 'country': 'first', 'year': 'first'}).rename(columns = {'begin_week': 'ds'})
holidays_weekly_de = holidays_weekly.query("(country == 'DE')").copy()
holidays_weekly_de

,ds,holiday,country,year
12,1994-12-26,Neujahr,DE,1995
183,1995-04-10,Karfreitag,DE,1995
222,1995-04-17,Ostermontag,DE,1995
270,1995-05-01,Erster Mai,DE,1995
346,1995-05-22,Christi Himmelfahrt,DE,1995
...,...,...,...,...
35445,2044-05-23,Christi Himmelfahrt,DE,2044
35481,2044-06-06,Pfingstmontag,DE,2044
35646,2044-10-03,Tag der Deutschen Einheit,DE,2044
35781,2044-12-19,Erster Weihnachtstag,DE,2044


In [5]:
data = pd.read_csv("./data/data_raw_2015-11-23__2019-11-11.csv", parse_dates = ["DATE"])
data.columns = [c.lower() if c in ["DATE"] else c for c in data.columns]
data

,date,revenue,tv_S,ooh_S,print_S,facebook_I,search_clicks_P,search_S,competitor_sales_B,facebook_S,events,newsletter
0,2015-11-23,2.754372e+06,167687.6,0,95463.666667,7.290385e+07,0.000000,0,8125009,228213.987444,na,19401.653846
1,2015-11-30,2.584277e+06,214600.9,0,0.000000,1.658110e+07,29511.715457,31000,7901549,34258.573511,na,14791.000000
2,2015-12-07,2.547387e+06,0.0,248022,3404.000000,4.995477e+07,36132.358958,28400,8300197,127691.261335,na,14544.000000
3,2015-12-14,2.875220e+06,625877.3,0,132600.000000,3.164930e+07,36804.210958,31900,8122883,84014.720306,na,2800.000000
4,2015-12-21,2.215953e+06,0.0,520005,0.000000,8.802269e+06,28401.744069,27100,7105985,20687.478156,na,15478.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
203,2019-10-14,2.456240e+06,0.0,60433,153723.666667,0.000000e+00,152840.323412,112100,7612762,0.000000,na,28157.000000
204,2019-10-21,2.182825e+06,154917.6,0,0.000000,5.688196e+07,103680.047821,103700,6701667,133624.575524,na,10644.000000
205,2019-10-28,2.377707e+06,21982.5,14094,17476.000000,0.000000e+00,138387.704138,114700,7688920,0.000000,na,9597.000000
206,2019-11-04,2.732825e+06,22453.0,0,24051.333333,0.000000e+00,151707.990462,134100,8815710,0.000000,na,90189.000000


## Prophet Decomposition

In [6]:
prophet_data = data.rename(columns = {'revenue': 'y', 'date': 'ds'})
#add categorical into prophet
prophet_data = pd.concat([prophet_data, pd.get_dummies(prophet_data["events"], drop_first = True, prefix = "events")], axis = 1)
prophet_data


prophet = Prophet(yearly_seasonality=True, holidays=holidays_weekly_de)
prophet.add_regressor(name = "events_event2")
prophet.add_regressor(name = "events_na")


prophet.fit(prophet_data[["ds", "y", "events_event2", "events_na"]])
prophet_predict = prophet.predict(prophet_data[["ds", "y", "events_event2", "events_na"]])


prophet_columns = [col for col in prophet_predict.columns if (col.endswith("upper") == False) & (col.endswith("lower") == False)]
events_numeric = prophet_predict[prophet_columns].filter(like = "events_").sum(axis = 1)


final_data = data.copy()
final_data["trend"] = prophet_predict["trend"]
final_data["season"] = prophet_predict["yearly"]
final_data["holiday"] = prophet_predict["holidays"]
final_data["events"] = (events_numeric - np.min(events_numeric)).values

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [7]:
final_data.head()

,date,revenue,tv_S,ooh_S,print_S,facebook_I,search_clicks_P,search_S,competitor_sales_B,facebook_S,events,newsletter,trend,season,holiday
0,2015-11-23,2.754372e+06,167687.6,0,95463.666667,7.290385e+07,0.000000,0,8125009,228213.987444,0.0,19401.653846,2.890375e+06,1.126061e+06,0.000000
1,2015-11-30,2.584277e+06,214600.9,0,0.000000,1.658110e+07,29511.715457,31000,7901549,34258.573511,0.0,14791.000000,2.891399e+06,8.988662e+05,0.000000
2,2015-12-07,2.547387e+06,0.0,248022,3404.000000,4.995477e+07,36132.358958,28400,8300197,127691.261335,0.0,14544.000000,2.892422e+06,7.142271e+05,0.000000
3,2015-12-14,2.875220e+06,625877.3,0,132600.000000,3.164930e+07,36804.210958,31900,8122883,84014.720306,0.0,2800.000000,2.893446e+06,6.939497e+05,0.000000
4,2015-12-21,2.215953e+06,0.0,520005,0.000000,8.802269e+06,28401.744069,27100,7105985,20687.478156,0.0,15478.000000,2.894470e+06,7.892366e+05,502772.522021


## Line Fit

In [8]:
target = "revenue"
media_channels = ["tv_S", "ooh_S", "print_S", "facebook_S", "search_S"]
organic_channels = ["newsletter"]
control_features = ["trend", "season", "holiday", "competitor_sales_B", "events"]
features = control_features + media_channels + organic_channels

## Ordinary Least Squares (Unbounded)

In [9]:
data_with_intercept = tools.add_constant(final_data[features])
ols = linear_model.OLS(endog = data[target], exog = data_with_intercept)
ols_res = ols.fit()

In [10]:
print(ols_res.summary())

                            OLS Regression Results                            
Dep. Variable:                revenue   R-squared:                       0.957
Model:                            OLS   Adj. R-squared:                  0.955
Method:                 Least Squares   F-statistic:                     400.6
Date:                Wed, 22 Jun 2022   Prob (F-statistic):          7.50e-128
Time:                        18:48:15   Log-Likelihood:                -2770.6
No. Observations:                 208   AIC:                             5565.
Df Residuals:                     196   BIC:                             5605.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const              -2.351e+06   1.15

In [11]:
print(ols_res.params)

const                -2.350707e+06
trend                 8.816014e-01
season                1.694433e-01
holiday               9.653860e-01
competitor_sales_B    2.584796e-01
events                1.088028e+00
tv_S                  3.230263e-01
ooh_S                 1.096624e-01
print_S               8.584973e-01
facebook_S            5.933667e-02
search_S             -7.370236e-02
newsletter            1.175328e+00
dtype: float64


In [12]:
len(features)

11

## Curve Fit: Non-Linear Least Squares

In [13]:
def linear_function(data, a, b1, b2, b3, b4, 
                          b5, b6, b7, b8, b9, b10, b11):
    return a + b1 * data[:, 0]\
             + b2 * data[:, 1]\
             + b3 * data[:, 2]\
             + b4 * data[:, 3]\
             + b5 * data[:, 4]\
             + b6 * data[:, 5]\
             + b7 * data[:, 6]\
             + b8 * data[:, 7]\
             + b9 * data[:, 8]\
             + b10 * data[:, 9]\
             + b11 * data[:, 10]

In [14]:
curve_fit_coefs, _ = curve_fit(f = linear_function, 
                               method = "lm", 
                               xdata=final_data[features].values,
                               ydata = final_data[target].values)

In [15]:
pd.DataFrame(curve_fit_coefs, index=["const"] + features, columns=["coefficient"])

,coefficient
const,-2.350708e+06
trend,8.816017e-01
season,1.694432e-01
holiday,9.653860e-01
competitor_sales_B,2.584796e-01
events,1.088028e+00
tv_S,3.230262e-01
ooh_S,1.096624e-01
print_S,8.584974e-01
facebook_S,5.933667e-02


### Lower/Upper Bounds for curve_fit

In [16]:
def prepare_bounds(intercept_value, 
                         control_length, 
                         media_length, 
                         control_value, 
                         media_value):
    lower_bounds_array = []
    lower_bounds_array.append(intercept_value)

    for i in range(control_length):
        lower_bounds_array.append(control_value)

    for i in range(media_length):
        lower_bounds_array.append(media_value)
        
    return lower_bounds_array

### Bounds for curve_fit

In [17]:
lower_bounds_array = \
    prepare_bounds(intercept_value = 0, 
                   control_length = len(control_features), 
                   media_length = len(media_channels) + len(organic_channels), 
                   control_value = -np.inf, 
                   media_value = 0)
upper_bounds_array = \
    prepare_bounds(intercept_value = np.inf, 
                   control_length = len(control_features), 
                   media_length = len(media_channels) + len(organic_channels), 
                   control_value = np.inf, 
                   media_value = np.inf)
curve_fit_bounded_coefs, _ = \
    curve_fit(f = linear_function,
          xdata=final_data[features].values, 
          ydata = final_data[target].values,
          bounds = (tuple(lower_bounds_array), 
                    tuple(upper_bounds_array)))

In [18]:
lower_bounds_array

[0, -inf, -inf, -inf, -inf, -inf, 0, 0, 0, 0, 0, 0]

In [19]:
upper_bounds_array

[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]

In [20]:
pd.DataFrame(curve_fit_bounded_coefs, index = ["const"] + features, columns = ["coefficient"])

,coefficient
const,4.371483e-13
trend,8.438923e-02
season,1.720417e-01
holiday,9.603329e-01
competitor_sales_B,2.565984e-01
events,1.099378e+00
tv_S,3.351958e-01
ooh_S,1.306090e-01
print_S,8.482113e-01
facebook_S,5.604416e-02


In [21]:
final_data[features].values @ curve_fit_bounded_coefs[1:] + curve_fit_bounded_coefs[0]

array([2694867.03585063, 2517089.04041769, 2556001.52024827,
       2778086.06659595, 2773192.0553033 , 2737277.26125289,
       2259024.71043342, 2437194.47043615, 2055767.35169091,
       1803821.14202479, 1751158.83665983, 1828642.1334421 ,
       1959172.29966295, 1551180.44899551, 1497524.74359224,
       1592706.63664419, 1510993.01791504, 1352201.0808354 ,
       2388018.25463024, 1121299.25919079, 1117010.63756754,
       1087805.18127523, 1204817.60341273,  852818.66003292,
        880567.83517686, 1259901.0348051 , 1003225.22188927,
        831919.50467648,  692644.18607075,  729100.1734281 ,
        957991.89277816,  840511.74071893, 1075916.00612412,
       1116614.49261775, 1266492.46230636, 1312600.71684545,
       1471455.86591771, 1503131.91925694, 1529357.97223463,
       1476922.23625193, 1931663.07062811, 2079114.98225395,
       2253831.87537573, 2229817.28190234, 1958029.48074046,
       2501700.78818646, 2449699.7151463 , 2360419.19028616,
       2663402.87049517,

In [22]:
final_data[target].values

array([2754371.66666667, 2584276.66666667, 2547386.66666667,
       2875220.        , 2215953.33333333, 2569921.66666667,
       2171506.66666667, 2464131.66666667, 2012520.        ,
       1738911.66666667, 1772306.66666667, 1809058.33333333,
       1952740.        , 1507805.        , 1510391.66666667,
       1588840.        , 1605990.        , 1356010.        ,
       2103936.66666667, 1120835.        , 1141140.        ,
       1166880.        ,  888806.66666667,  898873.33333333,
        889373.33333333, 1915375.        , 1006845.        ,
        869198.33333333,  727250.        ,  753730.        ,
        999196.66666667,  845843.33333333, 1126016.66666667,
       1140978.33333333, 1242656.66666667, 1293483.33333333,
       1484843.33333333, 1485698.33333333, 1556608.33333333,
       1391206.66666667, 2018783.33333333, 2170721.66666667,
       2247670.        , 2192026.66666667, 1895213.33333333,
       2232910.        , 2406035.        , 2252445.        ,
       2663125.        ,

## Ridge Wrapper with rpy2

In [23]:
from rpy2 import robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
import rpy2.situation
base = importr("base")
glm = importr("glmnet")

In [24]:
for row in rpy2.situation.iter_info():
    print(row)

rpy2 version:
3.4.5
Python version:
3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]
Looking for R's HOME:
    Environment variable R_HOME: None
    InstallPath in the registry: C:\Program Files\R\R-4.1.2
    Environment variable R_USER: None
    Environment variable R_LIBS_USER: None
R version:
    In the PATH: 
    Loading R library from rpy2: OK
Additional directories to load R packages from:
None
C extension compilation:
  include:
  ['C:/PROGRA~1/R/R-41~1.2/include', 'C:/PROGRA~1/R/R-41~1.2/include/x64']
  libraries:
  ['R', 'm']
  library_dirs:
  ['C:/PROGRA~1/R/R-41~1.2/bin/x64']
  extra_compile_args:
  []
  extra_link_args:
  []


In [25]:
#R
data_frame_to_r_matrix = ro.r('function(x) as.matrix(x)')

run_cv_glmnet_in_r = \
    ro.r("""
   
#pure R code
function(x_train, 
        y_train, 
        is_intercept, 
        lower_limits, 
        upper_limits, 
        alpha = 0){ 
     cvmod = cv.glmnet(x_train,
                       y_train,
                       family = 'gaussian',
                       alpha = alpha,
                       lower.limits = lower_limits,
                       upper.limits = upper_limits,
                       type_measure = 'mse', 
                       intercept = is_intercept, 
                       nfolds = nrow(x_train))

     lambda_min = cvmod$lambda.min
     lambda_1se = cvmod$lambda.1se
     coefs = as.data.frame(as.matrix(coef(cvmod)))
     names(coefs)[1] = "value"

     return (list(coefs = coefs, lambda_min = lambda_min, lambda_best = lambda_1se))
    }

""")

run_glmnet_in_r = \
    ro.r("""
    
#pure R code
function(x_train, 
        y_train, 
        x_test, 
        lambda, 
        is_intercept, 
        lower_limits, 
        upper_limits, 
        alpha = 0){ 
     mod = glmnet(x_train,
                   y_train,
                   family = 'gaussian',
                   alpha = alpha,
                   lambda = lambda, 
                   intercept = is_intercept,
                   lower.limits = lower_limits,
                   upper.limits = upper_limits,
                   type_measure = 'mse')

     coefs = as.data.frame(as.matrix(coef(mod)))
     names(coefs)[1] = "value"
     y_pred = predict(mod, s = lambda, newx = x_test)
     return (list(coefs = coefs, 
                  y_pred = y_pred))
}

""")


def convert_r_list_into_dictionary(r_list):
    py_dict = {}
    #for name in coefs.names:
    keys = r_list.names
    for i in range(len(keys)):
        if isinstance(r_list[i], ro.vectors.DataFrame):
            with ro.conversion.localconverter(ro.default_converter + pandas2ri.converter):
                py_dict[keys[i]]=r_list[i]
        elif isinstance(r_list[i], ro.vectors.FloatVector):
            array = np.array(r_list[i])
            if len(array) == 1:
                array = array[0]
            py_dict[keys[i]] = array
        else:
            py_dict[keys[i]] = r_list[i]
    return py_dict


# Python

def run_cv_glmnet(x_train, y_train, is_intercept, lower_limits, upper_limits, alpha = 0):
    with ro.conversion.localconverter(ro.default_converter + pandas2ri.converter):
        r_df = ro.conversion.py2rpy(x_train)
        
    r_x_train = data_frame_to_r_matrix(r_df)
    r_y_train = ro.vectors.FloatVector(y_train)
    r_lower_limits = ro.vectors.FloatVector(lower_limits)
    r_upper_limits = ro.vectors.FloatVector(upper_limits)
    cvmod = run_cv_glmnet_in_r(r_x_train, r_y_train, ro.vectors.BoolVector([is_intercept]), r_lower_limits, r_upper_limits, alpha)
    cvmod = convert_r_list_into_dictionary(cvmod)
    cvmod["coefs"] = cvmod["coefs"].reset_index()
    return cvmod

def run_glmnet(x_train, 
               y_train, 
               x_test, 
               lambda_best, 
               is_intercept, 
               lower_limits, 
               upper_limits, 
               alpha = 0):
    #type conversions
    with ro.conversion.localconverter(ro.default_converter + pandas2ri.converter):
        r_df = ro.conversion.py2rpy(x_train)
        r_test_df = ro.conversion.py2rpy(x_test)
        
    r_x_train = data_frame_to_r_matrix(r_df)
    r_x_test = data_frame_to_r_matrix(r_test_df)
    
    r_y_train = ro.vectors.FloatVector(y_train)
    
    r_lower_limits = ro.vectors.FloatVector(lower_limits)
    r_upper_limits = ro.vectors.FloatVector(upper_limits)
    
    lambda_best_float = ro.vectors.FloatVector([lambda_best])
    
    is_intercept_bool = ro.vectors.BoolVector([is_intercept])
    
    #call glmnet
    mod = run_glmnet_in_r(r_x_train, 
                          r_y_train, 
                          r_x_test, 
                          lambda_best_float, 
                          is_intercept_bool, 
                          r_lower_limits, 
                          r_upper_limits, 
                          alpha)
    
    mod = convert_r_list_into_dictionary(mod)
    
    mod["coefs"] = mod["coefs"].reset_index()
    
    return mod

In [26]:
 # Set lower and upper bounds for feature coefficients
lower_limits = np.array([-np.inf for _ in range(len(control_features))] + [0 for _ in range(len(media_channels) + len(organic_channels))])
upper_limits = np.array([np.inf for _ in range(len(control_features))] + [np.inf for _ in range(len(media_channels) + len(organic_channels))])

print(lower_limits)
print(upper_limits)

[-inf -inf -inf -inf -inf   0.   0.   0.   0.   0.   0.]
[inf inf inf inf inf inf inf inf inf inf inf]


In [27]:
cv_glmnet = run_cv_glmnet(x_train = final_data[features], 
                          y_train = final_data[target], 
                          is_intercept = True, 
                          lower_limits = lower_limits , 
                          upper_limits = upper_limits)

In [28]:
cv_glmnet["coefs"]

,index,value
0,(Intercept),-1.532470e+06
1,trend,7.994577e-01
2,season,3.747886e-01
3,holiday,7.217806e-01
4,competitor_sales_B,1.415311e-01
5,events,9.291297e-01
6,tv_S,3.577095e-01
7,ooh_S,1.311229e-01
8,print_S,8.445607e-01
9,facebook_S,2.933003e-01


In [29]:
cv_glmnet["lambda_min"]

65481.09025336275

In [30]:
cv_glmnet["lambda_best"]

166018.4312548283

In [31]:
results = run_glmnet(x_train = final_data[features], 
                     y_train = final_data[target], 
                     x_test = final_data[features], 
                     lambda_best = cv_glmnet["lambda_best"], 
                     is_intercept = True, 
                     lower_limits = lower_limits, 
                     upper_limits = upper_limits)

In [32]:
results.keys()

dict_keys(['coefs', 'y_pred'])

In [33]:
results["coefs"]

,index,value
0,(Intercept),-1.531885e+06
1,trend,7.990260e-01
2,season,3.742805e-01
3,holiday,7.218073e-01
4,competitor_sales_B,1.416375e-01
5,events,9.290838e-01
6,tv_S,3.578114e-01
7,ooh_S,1.312565e-01
8,print_S,8.447765e-01
9,facebook_S,2.936188e-01


In [34]:
results["y_pred"]

array([[2594013.22443688],
       [2380150.61719424],
       [2351297.24299364],
       [2588586.69648715],
       [2576609.71959664],
       [2656899.8420413 ],
       [2221632.42706163],
       [2305562.54497911],
       [1961206.32770898],
       [1815695.60904306],
       [1736528.25809711],
       [1781941.17263507],
       [1883178.9131058 ],
       [1523275.10776326],
       [1527139.03662875],
       [1532458.21241944],
       [1433857.61205615],
       [1302851.12995012],
       [2153362.8891416 ],
       [1149562.32740569],
       [1112825.91504463],
       [1087945.21337724],
       [1156553.0127093 ],
       [ 920914.48987567],
       [ 947713.69342941],
       [1272970.41292986],
       [1026352.93952706],
       [ 872264.59624444],
       [ 766618.79354409],
       [ 810041.38584948],
       [1026795.51308837],
       [ 926772.32958347],
       [1097682.0240212 ],
       [1120113.82362868],
       [1322754.83017775],
       [1336467.32970482],
       [1549799.09064877],
 